In [71]:
# Import necessary functions
import os
import aryn_sdk
from aryn_sdk.partition import partition_file, draw_with_boxes, convert_image_element
import json
import pdf2image
#from openai import OpenAI
from pydantic import BaseModel
#from openai.lib._parsing import type_to_response_format_param as pydantic_to_response_format
#from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

True

In [72]:
# Set your secrets in the colab notebook. Navigate to the left pane
# and choose the key option to set your keys. Make sure to enable Notebook access

# Visit https://www.aryn.ai/get-started to get a key.
aryn_api_key = os.getenv('aryn_API_KEY')
# Visit https://platform.openai.com/api-keys to get a key.
openai_api_key = os.getenv('anthropic_API_KEY')

In [73]:
file_name = 'syllabus206.pdf'
file = open(file_name, 'rb')

#show the pdf
#pdf2image.convert_from_path(file_name)[0]

In [74]:
## Make a call to the partitioning service and set extract_images to true.
partitioned_file = partition_file(file, aryn_api_key, extract_images=True, extract_table_structure=True, use_ocr=True)

# show the pdf with bounding boxes
#draw_with_boxes(file_name, partitioned_file)[0]

ArynPartitioner: T+   0.00: Server version aryn-partitioner-0.20241226.161207 Model version 1.4
ArynPartitioner: T+   0.00: Received request with aryn_call_id=cc9a63f0-e0a2-445d-88ba-d68d2653e90c
ArynPartitioner: T+   0.00: Waiting for scheduling
ArynPartitioner: T+   0.00: Preprocessing document
ArynPartitioner: T+   0.00: Done preprocessing document
ArynPartitioner: T+   0.75: Completed work on page 2
ArynPartitioner: T+   0.82: Completed work on page 1


In [75]:
## Walk over the image elements in the returned JSON and show the first one
data = partitioned_file['elements']
print(data)

[{'type': 'Section-header', 'bbox': [0.28595584644990807, 0.09077858664772727, 0.7166172880284927, 0.12163728193803267], 'properties': {'score': 0.8352801203727722, 'page_number': 1}, 'text_representation': 'CS 206 - Scientific Computing'}, {'type': 'Text', 'bbox': [0.14615026137408088, 0.14065311778675427, 0.7507750028722426, 0.18089363791725852], 'properties': {'score': 0.7316327095031738, 'page_number': 1}, 'text_representation': "Prof: Wayne Hayes (office: DBH4092), whayes@ uci.edu Office Hours: To be negotiated, or by appointment (don't be shy)"}, {'type': 'Section-header', 'bbox': [0.1469548303940717, 0.20158973000266336, 0.814414852366728, 0.22124447909268466], 'properties': {'score': 0.30405187606811523, 'page_number': 1}, 'text_representation': 'Topics covered (subject to change with consent of professor + class)'}, {'type': 'Text', 'bbox': [0.1471503672880285, 0.22100622003728693, 0.8567066147748161, 0.34354655872691764], 'properties': {'score': 0.7956948280334473, 'page_numb

In [76]:
# Pydantic models used to form response_format parameter to anthropic
class description(BaseModel):
    image_title: str
    image_description: str

class diagram_response(BaseModel):
    steps: list[description]

In [77]:
diagram_response_schema = diagram_response.model_json_schema()

tools = [
    {
        "name": "build_description_result",
        "description": "build the description object",
        "input_schema": diagram_response_schema
    }
]

In [78]:
def format_content(structured_content):
    formatted_text = ""
    for item in structured_content:
        if item.get("type") in ["Section-header", "Text", "List-item"]:
            formatted_text += f"{item['text_representation']}\n\n"
        elif item.get("type") == "table":
            formatted_text += "Table:\n"
            for cell in item['table']['cells']:
                formatted_text += f"{cell['content']} | "
            formatted_text += "\n\n"
    return formatted_text

formatted_content = format_content(data)
print(data)

[{'type': 'Section-header', 'bbox': [0.28595584644990807, 0.09077858664772727, 0.7166172880284927, 0.12163728193803267], 'properties': {'score': 0.8352801203727722, 'page_number': 1}, 'text_representation': 'CS 206 - Scientific Computing'}, {'type': 'Text', 'bbox': [0.14615026137408088, 0.14065311778675427, 0.7507750028722426, 0.18089363791725852], 'properties': {'score': 0.7316327095031738, 'page_number': 1}, 'text_representation': "Prof: Wayne Hayes (office: DBH4092), whayes@ uci.edu Office Hours: To be negotiated, or by appointment (don't be shy)"}, {'type': 'Section-header', 'bbox': [0.1469548303940717, 0.20158973000266336, 0.814414852366728, 0.22124447909268466], 'properties': {'score': 0.30405187606811523, 'page_number': 1}, 'text_representation': 'Topics covered (subject to change with consent of professor + class)'}, {'type': 'Text', 'bbox': [0.1471503672880285, 0.22100622003728693, 0.8567066147748161, 0.34354655872691764], 'properties': {'score': 0.7956948280334473, 'page_numb

In [79]:
import anthropic

client = anthropic.Anthropic(api_key=openai_api_key)
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                # {
                #     "type": "text",
                #     "source": {
                #         "type": "text",
                #         "media_type": "text",
                #         "data": f"{formatted_content}",
                #     },
                # },
                {
                    "type": "text",
                    "text": f"Describe this syllabus. {formatted_content}"
                }
            ],
        }
    ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_description_result"}

)
print(message)


Message(id='msg_019Jadt9tt7Xyb1arCgod4EW', content=[ToolUseBlock(id='toolu_014EAeAExsDZyr6wU1GCyLbB', input={'steps': [{'image_title': 'Course Overview & Instructor', 'image_description': 'CS 206 - Scientific Computing is taught by Prof. Wayne Hayes (office: DBH4092). Office hours are negotiable and appointments are welcome.'}, {'image_title': 'Course Topics', 'image_description': "The course covers comprehensive topics in scientific computing including: floating point numbers, Taylor series, condition numbers, numerical differentiation/integration, linear systems, eigenvalues, least squares fitting, optimization methods, Newton's method, gradient descent, interpolation, differential equations, chaos, and Monte Carlo methods."}, {'image_title': 'Grading Structure', 'image_description': 'The course grade is divided into three main components: 6-8 Homework Assignments (40%), Midterm Exam (20%), and Final Project with write-up (40%). Homework is due weekly except during midterm week.'}, {

In [80]:
print(json.dumps(message.content[0].input, indent=4))

{
    "steps": [
        {
            "image_title": "Course Overview & Instructor",
            "image_description": "CS 206 - Scientific Computing is taught by Prof. Wayne Hayes (office: DBH4092). Office hours are negotiable and appointments are welcome."
        },
        {
            "image_title": "Course Topics",
            "image_description": "The course covers comprehensive topics in scientific computing including: floating point numbers, Taylor series, condition numbers, numerical differentiation/integration, linear systems, eigenvalues, least squares fitting, optimization methods, Newton's method, gradient descent, interpolation, differential equations, chaos, and Monte Carlo methods."
        },
        {
            "image_title": "Grading Structure",
            "image_description": "The course grade is divided into three main components: 6-8 Homework Assignments (40%), Midterm Exam (20%), and Final Project with write-up (40%). Homework is due weekly except during mid